# Question 1

### Scrape Wikipedia page to generate a dataframe



In [97]:
import pandas as pd

In [ ]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada.html', 'w') as fo:
    fo.write(article)

In [90]:
from bs4 import BeautifulSoup

article = open('List_of_postal_codes_of_Canada.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break
headings = ['Postal code', 'Borough', 'Neighbourhood']
with open('postal_codes_canada.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        code, borough, neigh = [td.text.strip() for td in tds[:3]]        
        if 'Not assigned' in borough:
            continue
        elif 'Not assigned' in neigh:
            neigh = borough        
        print(';'.join([code, borough, neigh]), file=fo)
        
data = pd.read_csv('postal_codes_canada.txt', sep=";",header=None)
data.columns = headings
data.head(10)
         

,Postal code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


###### The dataframe looks like above

We now apply groupby to combine the neighbourhood names and find the shape

In [91]:
data = data.groupby(['Postal code','Borough'])['Neighbourhood'].apply(','.join).reset_index()
data.head(10)



,Postal code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [92]:
data.dtypes

Postal code      object
Borough          object
Neighbourhood    object
dtype: object

In [93]:
data.shape

(103, 3)

# Question 2

##### Adding new columns Latitude and Longitude by reading data from csv


In [94]:
data_gc = pd.read_csv('https://cocl.us/Geospatial_data')

data_gc.head(10)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [95]:
datamaster = data.set_index('Postal code').join(data_gc.set_index('Postal Code'))
datamaster.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postal code,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [96]:
datamaster.reset_index()

,Postal code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Question 3


In [111]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

In [116]:
# just copying my datamaster into neighborhoods
neighborhoods = datamaster
neighborhoods



,Borough,Neighbourhood,Latitude,Longitude
Postal code,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


## I will first plot a Folium Map for Toronto

## Next I will plot the map for York

## I will then pick up a neighbourhood in York and explore the place using the explore venue option

In [117]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Folium Map of Toronto

In [118]:
# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Folium Map of York

In [135]:
#slice data of York

york_data = neighborhoods[neighborhoods['Borough'] == 'York'].reset_index(drop=True)
york_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,York,Humewood-Cedarvale,43.693781,-79.428191
1,York,Caledonia-Fairbanks,43.689026,-79.453512
2,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",43.691116,-79.476013
3,York,"The Junction North,Runnymede",43.673185,-79.487262
4,York,Weston,43.706876,-79.518188


In [129]:
latitude = 43.693781
longitude = -79.428191

In [130]:
# create map  using latitude and longitude values
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

# Explore the neighbourhood of Weston

In [138]:
lat= 43.706876
long= -79.518188
VEERSION = '20190809'
radius = 800
LIMIT = 500

In [137]:
CLIENT_ID = 'YBT4LLDV1IP0XRTAUK1BD5OZY4GBHWXTDOKMQFCT0I2YR3XB' # your Foursquare ID
CLIENT_SECRET = 'G1QSFA0ECSZYGSHNAATC3Z25ZNRDKGA0DF0ZL1CBHPUABOPC' # your Foursquare Secret
VERSION = '20190718' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YBT4LLDV1IP0XRTAUK1BD5OZY4GBHWXTDOKMQFCT0I2YR3XB
CLIENT_SECRET:G1QSFA0ECSZYGSHNAATC3Z25ZNRDKGA0DF0ZL1CBHPUABOPC


In [139]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=YBT4LLDV1IP0XRTAUK1BD5OZY4GBHWXTDOKMQFCT0I2YR3XB&client_secret=G1QSFA0ECSZYGSHNAATC3Z25ZNRDKGA0DF0ZL1CBHPUABOPC&ll=43.706876,-79.518188&v=20190718&radius=800&limit=500'

In [150]:
import requests
results = requests.get(url).json()
'There are {} venues.'.format(len(results['response']['groups'][0]['items']))

'There are 6 venues.'

In [151]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4c670e987abde21e0b6d6568',
  'name': 'P&M Restaurant',
  'location': {'address': '1972 Weston Road',
   'lat': 43.70110845129989,
   'lng': -79.5189817242212,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.70110845129989,
     'lng': -79.5189817242212}],
   'distance': 645,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['1972 Weston Road', 'Toronto ON', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d147941735',
    'name': 'Diner',
    'pluralName': 'Diners',
    'shortName': 'Diner',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4c670e987abde21e0b6d6568-0'}

In [152]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,P&M Restaurant,Diner,1972 Weston Road,CA,Toronto,Canada,NaN,645,"[1972 Weston Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.70110845129989...",43.701108,-79.518982,NaN,ON,4c670e987abde21e0b6d6568
1,Shoppers Drug Mart,Pharmacy,1995 Weston Rd,CA,Toronto,Canada,at Lawrence Ave. W.,636,"[1995 Weston Rd (at Lawrence Ave. W.), Toronto...","[{'label': 'display', 'lat': 43.701264, 'lng':...",43.701264,-79.519705,M9N 1X2,ON,4c3b9d8d5810a593b1f8ba3c
2,Pizza Pizza,Pizza Place,1937 Weston Rd.,CA,Toronto,Canada,NaN,702,"[1937 Weston Rd., Toronto ON M9N 1W7, Canada]","[{'label': 'display', 'lat': 43.70058873986015...",43.700589,-79.517499,M9N 1W7,ON,4cc2280b06c254812dfd8747
3,Popeyes Louisiana Kitchen,Fried Chicken Joint,1975 Weston Rd.,CA,Toronto,Canada,at Little Ave.,651,"[1975 Weston Rd. (at Little Ave.), Toronto ON ...","[{'label': 'display', 'lat': 43.70104530536844...",43.701045,-79.518921,M9N 1W8,ON,4e52ee7881dc6e249dd940aa
4,Greenland Farm Supermarket,Grocery Store,NaN,CA,NaN,Canada,NaN,589,[Canada],"[{'label': 'display', 'lat': 43.7016347647671,...",43.701635,-79.519201,NaN,NaN,4cb372c4eb65b1f74ec083cd
5,Eggsmart,Breakfast Spot,2097 Weston Rd,CA,Toronto,Canada,NaN,647,"[2097 Weston Rd, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.70227, 'lng': ...",43.702270,-79.523103,NaN,ON,581e1fbb52d40746138b2231


In [153]:
venues_map = folium.Map(location=[lat, long], zoom_start=15) # generate map centred around Weston


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map